In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch Dataset 

In [18]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../../../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))


ind = int(0.8 * len(index))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))
molecules = (molecules[:ind], molecules[ind:])



s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_triu_norm, mu, std = AbstractDataset.normalize(s_triu)


s_train, p_train, s_test, p_test = split(s_triu_norm, p_triu, ind)

# Pure blocks

i will now set all other elements zero

## Utilities

In [3]:
from SCFInitialGuess.utilities.constants import number_of_basis_functions as N_BASIS

mol = molecules[0][0]

mask = np.zeros((dim, dim))

current_dim = 0
for atom in mol.species:
    
    # calculate block range
    index_start = current_dim
    current_dim += N_BASIS["sto-3g"][atom] 
    index_end = current_dim
    
    # calculate logical vector
    L = np.arange(dim)
    L = np.logical_and(index_start <= L, L < index_end)
    
    m = np.logical_and.outer(L, L)
    mask = np.logical_or(mask, m)
    
#mask

In [ ]:
import seaborn as sns

sns.heatmap(mask.astype("int"))

In [27]:
from SCFInitialGuess.utilities.dataset import reconstruct_from_triu

def extract_pure(p):
    p_batch = reconstruct_from_triu(p.copy(), dim)
    p_batch[np.logical_not(mask)] = 0.0
    return p_batch

p_pure = np.array(list(map(extract_pure, p_test))).astype("float64")

In [ ]:
p_pure[0]

In [4]:
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities
from SCFInitialGuess.utilities.dataset import StaticDataset

N_ELECTRONS = 30

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=mu,
    std=std
)


## Pure Block Analysis 

In [28]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer


print(make_results_str(measure_all_quantities(
    p_pure,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-04 13:16:53: Iteration calculation: 0
[ ] 2018-10-04 13:16:54: Iteration calculation: 1
[ ] 2018-10-04 13:16:54: Iteration calculation: 2
[ ] 2018-10-04 13:16:55: Iteration calculation: 3
[ ] 2018-10-04 13:16:55: Iteration calculation: 4
[ ] 2018-10-04 13:16:56: Iteration calculation: 5
[ ] 2018-10-04 13:16:56: Iteration calculation: 6
[ ] 2018-10-04 13:16:57: Iteration calculation: 7
[ ] 2018-10-04 13:16:57: Iteration calculation: 8
[ ] 2018-10-04 13:16:57: Iteration calculation: 9
[ ] 2018-10-04 13:16:58: Iteration calculation: 10
[ ] 2018-10-04 13:16:58: Iteration calculation: 11
[ ] 2018-10-04 13:16:58: Iteration calculation: 12
[ ] 2018-10-04 13:16:59: Iteration calculation: 13
[ ] 2018-10-04 13:17:00: Iteration calculation: 14
[ ] 2018-10-04 13:17:00: Iteration calculation: 15
[ ] 2018-10-04 13:17:01: Iteration calculation: 16
[ ] 2018-10-04 13:17:01: Iteration calculation: 17
[ ] 2018-10-04 13:17:02: Iteration calculation: 18
[ ] 2018-10-04 13:17:02: Iteration calcul

[ ] 2018-10-04 13:18:20: Iteration calculation: 160
[ ] 2018-10-04 13:18:20: Iteration calculation: 161
[ ] 2018-10-04 13:18:21: Iteration calculation: 162
[ ] 2018-10-04 13:18:21: Iteration calculation: 163
[ ] 2018-10-04 13:18:21: Iteration calculation: 164
[ ] 2018-10-04 13:18:22: Iteration calculation: 165
[ ] 2018-10-04 13:18:22: Iteration calculation: 166
[ ] 2018-10-04 13:18:23: Iteration calculation: 167
[ ] 2018-10-04 13:18:23: Iteration calculation: 168
[ ] 2018-10-04 13:18:24: Iteration calculation: 169
[ ] 2018-10-04 13:18:24: Iteration calculation: 170
[ ] 2018-10-04 13:18:25: Iteration calculation: 171
[ ] 2018-10-04 13:18:25: Iteration calculation: 172
[ ] 2018-10-04 13:18:26: Iteration calculation: 173
[ ] 2018-10-04 13:18:26: Iteration calculation: 174
[ ] 2018-10-04 13:18:27: Iteration calculation: 175
[ ] 2018-10-04 13:18:27: Iteration calculation: 176
[ ] 2018-10-04 13:18:28: Iteration calculation: 177
[ ] 2018-10-04 13:18:28: Iteration calculation: 178
[ ] 2018-10-

# Embedding the Blocks In Classical Guesses

In [5]:
from pyscf.scf import hf

p_sad = np.array([
    hf.init_guess_by_atom(mol.get_pyscf_molecule()) for mol in molecules[1]
]).astype("float64")
p_gwh = np.array([
    hf.init_guess_by_wolfsberg_helmholtz(mol.get_pyscf_molecule()) for mol in molecules[1]
]).astype("float64")

In [31]:
from SCFInitialGuess.utilities.dataset import reconstruct_from_triu

def embedd(x):
    p = x[1].copy()
    p[mask] = reconstruct_from_triu(x[0].copy(), dim)[mask]
    return p

p_embedded_sad = np.array(list(map(embedd, zip(p_test, p_sad))))
p_embedded_gwh = np.array(list(map(embedd, zip(p_test, p_gwh))))

## SAD

In [32]:
print(make_results_str(measure_all_quantities(
    p_embedded_sad,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-04 13:20:33: Iteration calculation: 0
[ ] 2018-10-04 13:20:34: Iteration calculation: 1
[ ] 2018-10-04 13:20:35: Iteration calculation: 2
[ ] 2018-10-04 13:20:35: Iteration calculation: 3
[ ] 2018-10-04 13:20:36: Iteration calculation: 4
[ ] 2018-10-04 13:20:36: Iteration calculation: 5
[ ] 2018-10-04 13:20:37: Iteration calculation: 6
[ ] 2018-10-04 13:20:37: Iteration calculation: 7
[ ] 2018-10-04 13:20:38: Iteration calculation: 8
[ ] 2018-10-04 13:20:38: Iteration calculation: 9
[ ] 2018-10-04 13:20:38: Iteration calculation: 10
[ ] 2018-10-04 13:20:39: Iteration calculation: 11
[ ] 2018-10-04 13:20:39: Iteration calculation: 12
[ ] 2018-10-04 13:20:40: Iteration calculation: 13
[ ] 2018-10-04 13:20:41: Iteration calculation: 14
[ ] 2018-10-04 13:20:41: Iteration calculation: 15
[ ] 2018-10-04 13:20:41: Iteration calculation: 16
[ ] 2018-10-04 13:20:42: Iteration calculation: 17
[ ] 2018-10-04 13:20:42: Iteration calculation: 18
[ ] 2018-10-04 13:20:43: Iteration calcul

[ ] 2018-10-04 13:21:57: Iteration calculation: 160
[ ] 2018-10-04 13:21:58: Iteration calculation: 161
[ ] 2018-10-04 13:21:58: Iteration calculation: 162
[ ] 2018-10-04 13:21:59: Iteration calculation: 163
[ ] 2018-10-04 13:21:59: Iteration calculation: 164
[ ] 2018-10-04 13:22:00: Iteration calculation: 165
[ ] 2018-10-04 13:22:00: Iteration calculation: 166
[ ] 2018-10-04 13:22:00: Iteration calculation: 167
[ ] 2018-10-04 13:22:01: Iteration calculation: 168
[ ] 2018-10-04 13:22:01: Iteration calculation: 169
[ ] 2018-10-04 13:22:02: Iteration calculation: 170
[ ] 2018-10-04 13:22:02: Iteration calculation: 171
[ ] 2018-10-04 13:22:03: Iteration calculation: 172
[ ] 2018-10-04 13:22:03: Iteration calculation: 173
[ ] 2018-10-04 13:22:04: Iteration calculation: 174
[ ] 2018-10-04 13:22:05: Iteration calculation: 175
[ ] 2018-10-04 13:22:05: Iteration calculation: 176
[ ] 2018-10-04 13:22:06: Iteration calculation: 177
[ ] 2018-10-04 13:22:06: Iteration calculation: 178
[ ] 2018-10-

## GWH

In [33]:
print(make_results_str(measure_all_quantities(
    p_embedded_gwh,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-04 13:24:38: Iteration calculation: 0
[ ] 2018-10-04 13:24:39: Iteration calculation: 1
[ ] 2018-10-04 13:24:39: Iteration calculation: 2
[ ] 2018-10-04 13:24:40: Iteration calculation: 3
[ ] 2018-10-04 13:24:40: Iteration calculation: 4
[ ] 2018-10-04 13:24:40: Iteration calculation: 5
[ ] 2018-10-04 13:24:41: Iteration calculation: 6
[ ] 2018-10-04 13:24:41: Iteration calculation: 7
[ ] 2018-10-04 13:24:42: Iteration calculation: 8
[ ] 2018-10-04 13:24:42: Iteration calculation: 9
[ ] 2018-10-04 13:24:42: Iteration calculation: 10
[ ] 2018-10-04 13:24:43: Iteration calculation: 11
[ ] 2018-10-04 13:24:43: Iteration calculation: 12
[ ] 2018-10-04 13:24:44: Iteration calculation: 13
[ ] 2018-10-04 13:24:44: Iteration calculation: 14
[ ] 2018-10-04 13:24:45: Iteration calculation: 15
[ ] 2018-10-04 13:24:45: Iteration calculation: 16
[ ] 2018-10-04 13:24:46: Iteration calculation: 17
[ ] 2018-10-04 13:24:46: Iteration calculation: 18
[ ] 2018-10-04 13:24:46: Iteration calcul

[ ] 2018-10-04 13:25:46: Iteration calculation: 160
[ ] 2018-10-04 13:25:47: Iteration calculation: 161
[ ] 2018-10-04 13:25:47: Iteration calculation: 162
[ ] 2018-10-04 13:25:47: Iteration calculation: 163
[ ] 2018-10-04 13:25:48: Iteration calculation: 164
[ ] 2018-10-04 13:25:48: Iteration calculation: 165
[ ] 2018-10-04 13:25:48: Iteration calculation: 166
[ ] 2018-10-04 13:25:49: Iteration calculation: 167
[ ] 2018-10-04 13:25:49: Iteration calculation: 168
[ ] 2018-10-04 13:25:50: Iteration calculation: 169
[ ] 2018-10-04 13:25:50: Iteration calculation: 170
[ ] 2018-10-04 13:25:50: Iteration calculation: 171
[ ] 2018-10-04 13:25:51: Iteration calculation: 172
[ ] 2018-10-04 13:25:51: Iteration calculation: 173
[ ] 2018-10-04 13:25:52: Iteration calculation: 174
[ ] 2018-10-04 13:25:52: Iteration calculation: 175
[ ] 2018-10-04 13:25:52: Iteration calculation: 176
[ ] 2018-10-04 13:25:53: Iteration calculation: 177
[ ] 2018-10-04 13:25:53: Iteration calculation: 178
[ ] 2018-10-

# Test 

In [34]:
from SCFInitialGuess.utilities.dataset import make_matrix_batch
s_test_batch = make_matrix_batch(S[ind:], dim)

In [35]:
from SCFInitialGuess.nn.post_processing import multi_mc_wheeny
from SCFInitialGuess.utilities.dataset import make_matrix_batch

#del p_embedded_sad
#del p_sad
#del p_gwh
#del P
#del dataset
#del molecules
#del p_pure
#del s_train
#del p_train

f = lambda x: multi_mc_wheeny(x[0], x[1], 20)

p_embedded_gwh_mcw5 = np.array(list(map(f, zip(p_embedded_gwh, s_test_batch))))

In [36]:
from SCFInitialGuess.utilities.analysis import mf_initializer

print(make_results_str(measure_all_quantities(
    p_embedded_gwh_mcw5,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-04 13:32:25: Iteration calculation: 0
[ ] 2018-10-04 13:32:26: Iteration calculation: 1
[ ] 2018-10-04 13:32:26: Iteration calculation: 2
[ ] 2018-10-04 13:32:27: Iteration calculation: 3
[ ] 2018-10-04 13:32:27: Iteration calculation: 4
[ ] 2018-10-04 13:32:28: Iteration calculation: 5
[ ] 2018-10-04 13:32:28: Iteration calculation: 6
[ ] 2018-10-04 13:32:29: Iteration calculation: 7
[ ] 2018-10-04 13:32:29: Iteration calculation: 8
[ ] 2018-10-04 13:32:29: Iteration calculation: 9
[ ] 2018-10-04 13:32:30: Iteration calculation: 10
[ ] 2018-10-04 13:32:30: Iteration calculation: 11
[ ] 2018-10-04 13:32:31: Iteration calculation: 12
[ ] 2018-10-04 13:32:31: Iteration calculation: 13
[ ] 2018-10-04 13:32:32: Iteration calculation: 14
[ ] 2018-10-04 13:32:32: Iteration calculation: 15
[ ] 2018-10-04 13:32:33: Iteration calculation: 16
[ ] 2018-10-04 13:32:33: Iteration calculation: 17
[ ] 2018-10-04 13:32:34: Iteration calculation: 18
[ ] 2018-10-04 13:32:34: Iteration calcul

[ ] 2018-10-04 13:33:47: Iteration calculation: 160
[ ] 2018-10-04 13:33:47: Iteration calculation: 161
[ ] 2018-10-04 13:33:48: Iteration calculation: 162
[ ] 2018-10-04 13:33:48: Iteration calculation: 163
[ ] 2018-10-04 13:33:48: Iteration calculation: 164
[ ] 2018-10-04 13:33:49: Iteration calculation: 165
[ ] 2018-10-04 13:33:49: Iteration calculation: 166
[ ] 2018-10-04 13:33:49: Iteration calculation: 167
[ ] 2018-10-04 13:33:50: Iteration calculation: 168
[ ] 2018-10-04 13:33:50: Iteration calculation: 169
[ ] 2018-10-04 13:33:51: Iteration calculation: 170
[ ] 2018-10-04 13:33:51: Iteration calculation: 171
[ ] 2018-10-04 13:33:51: Iteration calculation: 172
[ ] 2018-10-04 13:33:52: Iteration calculation: 173
[ ] 2018-10-04 13:33:52: Iteration calculation: 174
[ ] 2018-10-04 13:33:53: Iteration calculation: 175
[ ] 2018-10-04 13:33:53: Iteration calculation: 176
[ ] 2018-10-04 13:33:54: Iteration calculation: 177
[ ] 2018-10-04 13:33:54: Iteration calculation: 178
[ ] 2018-10-